## Login en Flask

 El control de acceso a nuestra aplicación debe implementarse adecuadamente, ya que, en gran medida, de él depende parte de la seguridad de nuestra aplicación.

Es necesario crear un modelo que representa a los usuarios de la aplicación y el formulario de login.

### Flask-login

¿Qué nos ofrece Flask-login? Entre otras cosas:

- Almacenar el ID del usuario en la sesión y mecanismos para hacer login y logout.
- Restringir el acceso a ciertas vistas únicamente a los usuarios autenticados.
- Gestionar la funcionalidad Recuérdame para mantener la sesión incluso después de que el usuario cierre el navegador.
- Proteger el acceso a las cookies de sesión frente a terceros.

> pip install flask-login

In [ ]:
from flask_login import LoginManager
...
app = Flask(__name__)
app.config['SECRET_KEY'] = '7110c8ae51a4b5af97be6534caef90e4bb9bdcb3380af008f90b23a5d1616bf319bc298105da20fe'
login_manager = LoginManager(app)

Crear el modelo User

Esta clase representa a los usuarios de nuestra aplicación. Además, contiene toda la lógica para crear usuarios, guardar las contraseñas de modo seguro o verificar los passwords.

Un punto a favor de la extensión Flask-login es que te da libertad para definir tu clase para los usuarios. Esto hace posible que se pueda utilizar cualquier sistema de base de datos y que modifiquemos el modelo en función de las necesidades que vayan surgiendo. El único requisito indicado por Flask-login es que la clase usuario debe implementar las siguientes propiedades y métodos:

- `is_authenticated`: una propiedad que es True si el usuario se ha autenticado y False en caso contrario.
- `is_active`: una propiedad que indica si la cuenta del usuario está activa (True) o no (False). Es decisión tuya definir qué significa que una cuenta de usuario está activa. Por ejemplo, se ha verificado el email o no ha sido eliminada por un administrador. Por defecto, los usuarios de cuentas inactivas no pueden autenticarse.
- `is_anonymous`: una propiedad que vale False para los usuarios reales y True para los usuarios anónimos.
- `get_id()`: un método que devuelve un string (unicode en caso de Python 2) con el ID único del usuario. Si el ID del usuario fuera int o cualquier otro tipo, es tu responsabilidad convertirlo a string.

In [ ]:
from flask_login import UserMixin
from werkzeug.security import generate_password_hash, check_password_hash
class User(UserMixin):
    def __init__(self, id, name, email, password, is_admin=False):
        self.id = id
        self.name = name
        self.email = email
        self.password = generate_password_hash(password)
        self.is_admin = is_admin
    def set_password(self, password):
        self.password = generate_password_hash(password)
    def check_password(self, password):
        return check_password_hash(self.password, password)
    def __repr__(self):
        return '<User {}>'.format(self.email)

### Guardar las contraseñas

 Las contraseñas de los usuarios no las guardaremos en exactas. Esto sería un problema de seguridad. En su lugar, guardaremos un hash del password. Para ello, nos valdremos de la librería `werkzeug.security`

### Listado de usuarios

Por el momento como no tenemos base de datos los usuarios los almacenaremos en una lista de usuarios almacenada en memoria que llamaremos users (esto no es útil ya que los usuarios que guardemos se borrarán al reiniciar el servidor pero nos servirá para esta lección).

In [ ]:
users = []
def get_user(email):
    for user in users:
        if user.email == email:
            return user
    return None

### Cargar el modelo

¿Cómo podemos acceder en nuestro código al usuario cuyo ID se encuentra almacenado en sesión? Fácil, implementando un callback que será llamado por el método user_loader del objeto login_manager.

El callback toma como parámetro un string con el ID del usuario que se encuentra en sesión y debe devolver el correspondiente objeto User o None si el ID no es válido. No lances una excepción si no puedes devolver un usuario a partir del ID.

In [ ]:
@login_manager.user_loader
def load_user(user_id):
    for user in users:
        if user.id == int(user_id):
            return user
    return None

users hace referencia a la lista de usuarios que definimos en el módulo models.py. Recuerda importarla previamente antes de hacer uso de ella.

In [ ]:
from models import users

### Login de usuarios

Ahora toca el turno de crear el formulario para que los usuarios de nuestro blog se puedan autenticar (para poder comentar los posts). Vamos a dividir este proceso en tres fases: crear la clase del formulario, crear la plantilla HTML e implementar la vista que realiza el login.

### Clase para el formulario de login

In [ ]:
class LoginForm(FlaskForm):
    email = StringField('Email', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Recuérdame')
    submit = SubmitField('Login')

El campo remember_me es de tipo BooleanField. Deberás importarlo junto al resto de tipos que importamos en la lección anterior. Lo utilizaremos para dar la posibilidad al usuario de mantener la sesión incluso después de cerrar el navegador.

### Plantilla HTML para el formulario

In [ ]:
{% extends "base_template.html" %}
{% block title %}Login{% endblock %}
{% block content %}
    <div>
        <form action="" method="post" novalidate>
            {{ form.hidden_tag() }}
            <div>
                {{ form.email.label }}
                {{ form.email }}<br>
                {% for error in form.email.errors %}
                <span style="color: red;">{{ error }}</span>
                {% endfor %}
            </div>
            <div>
                {{ form.password.label }}
                {{ form.password }}<br>
                {% for error in form.password.errors %}
                <span style="color: red;">{{ error }}</span>
                {% endfor %}
            </div>
            <div>{{ form.remember_me() }} {{ form.remember_me.label }}</div>
            <div>
                {{ form.submit() }}
            </div>
        </form>
    </div>
    <div>¿No tienes cuenta? <a href="{{ url_for('show_signup_form') }}">Regístrate</a></div>
{% endblock %}


### La vista para realizar el login

Debemos implementar la vista que muestre el formulario de login y compruebe si las credenciales proporcionadas por el usuario son válidas o no. Añade la siguiente función al final del fichero run.py:

In [ ]:
from werkzeug.urls import url_parse

@app.route('/login', methods=['GET', 'POST'])
def login():
    if current_user.is_authenticated:
        return redirect(url_for('index'))
    form = LoginForm()
    if form.validate_on_submit():
        user = get_user(form.email.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, remember=form.remember_me.data)
            next_page = request.args.get('next')
            if not next_page or url_parse(next_page).netloc != '':
                next_page = url_for('index')
            return redirect(next_page)
    return render_template('login_form.html', form=form)

- En primer lugar comprobamos si el usuario actual ya está autenticado. Para ello nos valemos de la instancia current_user de Flask-login. El valor de current_user será un objeto usuario si este está autenticado (el que se obtiene en el callback user_loader) o un usuario anónimo en caso contrario. Si el usuario ya está autenticado no tiene sentido que se vuelva a loguear, por lo que lo redirigimos a la página principal.
- A continuación comprobamos si los datos enviados en el formulario son válidos. En ese caso, intentamos recuperar el usuario a partir del email con get_user().
- Si existe un usuario con dicho email y la contraseña coincide, procedemos a autenticar al usuario llamando al método login_user de Flask-login.
- Por último comprobamos si recibimos el parámetro next. Esto sucederá cuando el usuario ha intentado acceder a una página protegida pero no estaba autenticado. Por temas de seguridad, solo tendremos en cuenta dicho parámetro si la ruta es relativa. De este modo evitamos redirigir al usuario a un sitio fuera de nuestro dominio. Si no se recibe el parámetro next o este no contiene una ruta relativa, redirigimos al usuario a la página de inicio.

### Logout

Cerrar la sesión en la misma

In [ ]:
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))

<div>
    <a href="../Flask.py">
        <img src="../img/return.png" alt="return" title="return" width="75" style="float: left;" />
    </a>
    <a href="./NB05.ipynb">
        <img src="../img/forward.png" alt="forward" title="forward" width="75" style="float: right;" />
    </a>
</div>